In [1]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

Using TensorFlow backend.


In [2]:
df = pd.read_csv('gst_data1.csv', encoding='unicode_escape')

In [3]:
df = df[pd.notnull(df['Type'])]
print(df.head(10))

                                                Data                  Type
0  A contractor or supplier can commit fraud by k...  Fake Invoice Numbers
1  False invoices refer to invoices for goods o...  Fake Invoice Numbers
2  Knowingly typically is defined as:\n\nActual ...  Fake Invoice Numbers
3  GENERAL RED FLAGS OF FALSE, INFLATED OR DUPLIC...  Fake Invoice Numbers
4  Red flags of false invoices:\n\nNo receiving r...  Fake Invoice Numbers
5  Red flags of inflated invoices:\n\nInvoice pri...  Fake Invoice Numbers
6  Red flags of duplicate invoices:\n\nMultiple p...  Fake Invoice Numbers
7  Identify and interview all complainants and co...  Fake Invoice Numbers
8  Obtain the following documents and examine the...  Fake Invoice Numbers
9  Do due diligence background checks on the cont...  Fake Invoice Numbers


In [4]:
from sklearn.utils import shuffle
import numpy as np

import logging
import pandas as pd
from numpy import random
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup


In [30]:
#preprocessing text data
#replace [/(){}\[\]\|@,;] by space ' '
#remove stopwords
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')   
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(txt):
    txt = BeautifulSoup(txt, "lxml").text # HTML decoding
    txt = txt.lower() # lowercase text
    txt = REPLACE_BY_SPACE_RE.sub(' ', txt) # replace REPLACE_BY_SPACE_RE symbols by space in text
    txt = BAD_SYMBOLS_RE.sub('', txt) # delete symbols which are in BAD_SYMBOLS_RE from text
    txt = ' '.join(word for word in txt.split() if word not in STOPWORDS) # delete stopwors from text
    return txt

In [31]:
df = df.dropna()

In [32]:
df['Data'] = df['Data'].apply(clean_text)

In [33]:
X = df['Data']
y = df['Type']

In [34]:
#shuffle dataset 
X, y = shuffle(X, y)

In [35]:
train_size = int(len(df) * .7)
train_posts = X[:train_size]
train_tags = y[:train_size]
test_posts = X[train_size:]
test_tags = y[train_size:]

In [36]:
max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_posts) # only fit on train

In [37]:
x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

In [38]:
encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [39]:
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [40]:
batch_size = 32
epochs = 5

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [41]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [42]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 927 samples, validate on 103 samples
Epoch 1/5
927/927 [==============================] - 1s 651us/step - loss: 1.6180 - accuracy: 0.3463 - val_loss: 1.2395 - val_accuracy: 0.5049
Epoch 2/5
927/927 [==============================] - 0s 538us/step - loss: 0.8377 - accuracy: 0.7152 - val_loss: 0.9553 - val_accuracy: 0.6311
Epoch 3/5
927/927 [==============================] - 0s 536us/step - loss: 0.4153 - accuracy: 0.8501 - val_loss: 1.2284 - val_accuracy: 0.6117
Epoch 4/5
927/927 [==============================] - 0s 529us/step - loss: 0.2991 - accuracy: 0.8986 - val_loss: 1.3563 - val_accuracy: 0.6311
Epoch 5/5
927/927 [==============================] - 0s 528us/step - loss: 0.2553 - accuracy: 0.8975 - val_loss: 1.3750 - val_accuracy: 0.6311


In [43]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

442/442 [==============================] - 0s 54us/step
Test accuracy: 0.6402714848518372
